In [4]:
!pip3 install -U ucimlrepo

  Obtaining dependency information for ucimlrepo from https://files.pythonhosted.org/packages/22/47/9350b2eeeaef8c0fd3ec3505c8a0481b576845b3df0d71c76f989c23d3c6/ucimlrepo-0.0.6-py3-none-any.whl.metadata


In [51]:
import pandas as pd
from ucimlrepo import fetch_ucirepo, list_available_datasets
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [6]:
nursery = fetch_ucirepo(id=76) 

In [7]:
X = nursery.data.features 
y = nursery.data.targets 

In [47]:
df = pd.DataFrame(data=X, columns=nursery.data.feature_names)
df['target'] = y
df

,parents,has_nurs,form,children,housing,finance,social,health,target
0,usual,proper,complete,1,convenient,convenient,nonprob,recommended,recommend
1,usual,proper,complete,1,convenient,convenient,nonprob,priority,priority
2,usual,proper,complete,1,convenient,convenient,nonprob,not_recom,not_recom
3,usual,proper,complete,1,convenient,convenient,slightly_prob,recommended,recommend
4,usual,proper,complete,1,convenient,convenient,slightly_prob,priority,priority
...,...,...,...,...,...,...,...,...,...
12955,great_pret,very_crit,foster,more,critical,inconv,slightly_prob,priority,spec_prior
12956,great_pret,very_crit,foster,more,critical,inconv,slightly_prob,not_recom,not_recom
12957,great_pret,very_crit,foster,more,critical,inconv,problematic,recommended,spec_prior
12958,great_pret,very_crit,foster,more,critical,inconv,problematic,priority,spec_prior


In [48]:
df['target'].value_counts()

target
not_recom     4320
priority      4266
spec_prior    4044
very_recom     328
recommend        2
Name: count, dtype: int64

In [49]:
def isRecommend(val):
    if val == 'not_recom':
        return val
    return 'recommend'

df['target'] = df['target'].apply(isRecommend) 

In [50]:
df['target'].value_counts()

target
recommend    8640
not_recom    4320
Name: count, dtype: int64

In [57]:
df['health'].value_counts()

health
recommended    4320
priority       4320
not_recom      4320
Name: count, dtype: int64

In [20]:
df_encoded = pd.get_dummies(df, drop_first=True, dtype='int')
df_encoded

,parents_pretentious,parents_usual,has_nurs_improper,has_nurs_less_proper,has_nurs_proper,has_nurs_very_crit,form_completed,form_foster,form_incomplete,children_2,children_3,children_more,housing_critical,housing_less_conv,finance_inconv,social_problematic,social_slightly_prob,health_priority,health_recommended,target_recommend
0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1
1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1
2,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1
4,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12955,0,0,0,0,0,1,0,1,0,0,0,1,1,0,1,0,1,1,0,1
12956,0,0,0,0,0,1,0,1,0,0,0,1,1,0,1,0,1,0,0,0
12957,0,0,0,0,0,1,0,1,0,0,0,1,1,0,1,1,0,0,1,1
12958,0,0,0,0,0,1,0,1,0,0,0,1,1,0,1,1,0,1,0,1


In [27]:
X_encoded = df_encoded.drop('target_recommend', axis=1)
y_encoded = df_encoded['target_recommend']
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y_encoded, random_state=1)

In [44]:
model = RandomForestClassifier()

In [45]:
model.fit(X_train, y_train)

C:\Users\jnpre\anaconda3\Lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
C:\Users\jnpre\anaconda3\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
C:\Users\jnpre\anaconda3\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
C:\Users\jnpre\anaconda3\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  i

RandomForestClassifier()

In [46]:
model.score(X_test, y_test)

C:\Users\jnpre\anaconda3\Lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
C:\Users\jnpre\anaconda3\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
C:\Users\jnpre\anaconda3\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
C:\Users\jnpre\anaconda3\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  i

1.0

In [52]:
accuracy_score(y_test, model.predict(X_test))

C:\Users\jnpre\anaconda3\Lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
C:\Users\jnpre\anaconda3\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
C:\Users\jnpre\anaconda3\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
C:\Users\jnpre\anaconda3\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  i

1.0

In [56]:
sum(model.feature_importances_)


1.0

In [53]:
model.feature_importances_

array([0.00312308, 0.00302567, 0.00162921, 0.00191308, 0.00179743,
       0.00175535, 0.00242224, 0.00225011, 0.00224052, 0.00219164,
       0.00224485, 0.00204876, 0.00292742, 0.0031111 , 0.00408169,
       0.0029569 , 0.0031798 , 0.49422358, 0.46287757])

In [59]:
column_names = X_train.columns
# Create a dictionary to map column names to feature importances
importance_dict = dict(zip(column_names, model.feature_importances_))

# Print the feature importances with column names
for feature, importance in importance_dict.items():
    print(f"{feature}: {importance}")

parents_pretentious: 0.003123080210083774
parents_usual: 0.0030256678786798317
has_nurs_improper: 0.0016292089065903068
has_nurs_less_proper: 0.001913081652190765
has_nurs_proper: 0.0017974348481425953
has_nurs_very_crit: 0.001755354292354547
form_completed: 0.0024222442161741095
form_foster: 0.002250105718083523
form_incomplete: 0.0022405169175491522
children_2: 0.002191644937167532
children_3: 0.0022448482759547294
children_more: 0.0020487626046672637
housing_critical: 0.00292741646746382
housing_less_conv: 0.003111097325865373
finance_inconv: 0.0040816855271154215
social_problematic: 0.002956897765261251
social_slightly_prob: 0.0031797974077654744
health_priority: 0.49422358042761766
health_recommended: 0.46287757462127294
